In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
pdf_files = [
    "PDF_resources/cbt-strategies-guide.pdf"
]

In [3]:
docs = []
for path in pdf_files:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

In [4]:
# Split into chunks of ~1000 characters with 200 overlap
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunked_docs = splitter.split_documents(docs)
print(f"Total chunks: {len(chunked_docs)}")

Total chunks: 184


In [5]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [6]:
embedding_model = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

c:\Users\USER\Desktop\BOT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_chroma import Chroma

#Store all the embeddings
vectordb = Chroma.from_documents(documents=chunked_docs,embedding=embedding_model,persist_directory="chroma_db")

In [8]:
vectordb.similarity_search("CBT is based on the idea that our thoughts, feelings, and behaviors are all connected")

[Document(id='c857bdb7-7a89-4e9b-9dd3-310a8e2f0619', metadata={'keywords': 'South Central Mental Illness Research Education and Clinical Center', 'title': 'Cognitive Behavioral Therapy Strategies', 'author': 'South Central Mental Illness Research Education and Clinical Center', 'page': 11, 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'source': 'PDF_resources/cbt-strategies-guide.pdf', 'trapped': '/False', 'page_label': '12', 'nccl_app': 'PDF', 'producer': 'Adobe PDF Library 15.0', 'total_pages': 103, 'subject': 'Cognitive Behavioral Therapy Strategies', 'creationdate': '2019-09-26T14:54:21-05:00', 'moddate': '2022-11-22T15:09:46-06:00', 'nccl_status': 'Passed', 'nccl_standard': 'HHS (2018 Regulations);'}, page_content='STEPS OF CBT\n \nCognitive Behavioral Therapy helps people become more aware of the relationship between \ntheir thoughts, feelings, and emotions. \nStep 2 \nStep 3 \nStep 1 \nBecome aware of: \nAutomatic thoughts \nFeelings \nCore beliefs \nBehaviors \nPhysical reactio

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama3-8b-8192")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are a helpful and empathetic mental health therapeutic assistant. Use the following context from therapeutic documents to answer the user's question.
If the answer is not in the context, say so clearly.

Context:
{context}

Question:
{input}

Answer as if you're speaking kindly to someone seeking support:
"""
)


In [11]:
document_chain = create_stuff_documents_chain(llm,prompt)
retriever = vectordb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [12]:
reply=retrieval_chain.invoke({"input":"Hey"})
print(reply['answer'])

Hello there! I'm so glad you reached out for support. It takes a lot of courage to admit when we need help, and I'm here to listen and offer guidance.

It seems like you're working on building your confidence and self-esteem. That's amazing! It's great that you're acknowledging your strengths and focusing on being a good spouse, parent, friend, and person.

Can you tell me what's been going on that's making you feel like you need to work on these affirmations? Is there something specific that's been challenging you lately? I'm here to listen and offer support.


In [13]:
print(reply)

{'input': 'Hey', 'context': [Document(id='fff5e65f-79c0-43e2-bfea-d19121cfa972', metadata={'nccl_standard': 'HHS (2018 Regulations);', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2022-11-22T15:09:46-06:00', 'subject': 'Cognitive Behavioral Therapy Strategies', 'keywords': 'South Central Mental Illness Research Education and Clinical Center', 'title': 'Cognitive Behavioral Therapy Strategies', 'producer': 'Adobe PDF Library 15.0', 'page_label': '52', 'total_pages': 103, 'trapped': '/False', 'source': 'PDF_resources/cbt-strategies-guide.pdf', 'nccl_status': 'Passed', 'page': 51, 'nccl_app': 'PDF', 'creationdate': '2019-09-26T14:54:21-05:00', 'author': 'South Central Mental Illness Research Education and Clinical Center'}, page_content='52'), Document(id='c74b7659-ff54-4955-b545-81eb24065624', metadata={'author': 'South Central Mental Illness Research Education and Clinical Center', 'page': 26, 'nccl_status': 'Passed', 'trapped': '/False', 'creationdate': '2019-09-26T14:54:2